# Part 3. Enhancement
The RNN model used in Part 2 is a basic model to perform the task of sentiment classification. In
this section, you will design strategies to improve upon the previous model you have built. You are
required to implement the following adjustments:

1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same
way as model parameters) during the training process.
2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words
and train your model again.
3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and
stacking multiple layers if possible.
4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment
classification.
5. Further improve your model. You are free to use any strategy other than the above mentioned solutions. Changing hyper-parameters or stacking more layers is not counted towards
a meaningful improvement.


## Question 1

Instead of keeping the word embeddings fixed, now update the word embeddings (the same
way as model parameters) during the training process.

### Approach

We will use the same model as in part 2 notebook, but now we will also back propagate
the loss into the word embeddings itself. This will mean that as the model learns,
the word embeddings would also update, causing the encoding of the words to change.

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from common_utils import UNK_TOKEN,EMBEDDING_DIM,SAVE_DIR,VOCAB_PATH,EMBEDDING_MATRIX_PATH,WORD2IDX_PATH,IDX2WORD_PATH,tokenize, EmbeddingMatrix

In [3]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, word_embedding:np.ndarray):
        super(RNNModel, self).__init__()

        # Word2Vec embedding layer
        # freeze=False to enable updates to embeddings
        self.word2vec_embeddings = nn.Embedding.from_pretrained(torch.tensor(word_embedding), freeze=False)

        # RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Look up word embeddings
        x = self.word2vec_embeddings(x)

        # Pass through RNN
        x, _ = self.rnn(x)

        # Take the last hidden state
        x = x[:, -1, :]

        # Pass through fully connected layer
        x = self.fc(x)

        return x


hidden_dim = 128
output_dim = 1
# Load EmbeddingMatrix
w2v_model = EmbeddingMatrix.load()

# Create RNN model
model = RNNModel(w2v_model.vocab_size,
                 w2v_model.dimension,
                 hidden_dim,
                 output_dim,
                 w2v_model.embedding_matrix)

# Create optimizer
optimizer = optim.Adam(model.parameters())

## Question 2

### Approach

As discussed in part 1, we have mentioned 2 approaches to handling of the
OOV words. We will now demonstrate the first approach, which is to replace the OOV
words with a special token. We will replace the OOV words with a special token
`<UNK>`.

In [4]:
import json
from common_utils import EMBEDDING_DIM, EMBEDDING_MATRIX_PATH, IDX2WORD_PATH, UNK_TOKEN, WORD2IDX_PATH, load_glove_embeddings


w2v_model = EmbeddingMatrix.load()
extended_vocab = w2v_model.vocab
extended_vocab.add(UNK_TOKEN)

# mapping of words to indices and vice versa
word2idx = {word: idx for idx, word in enumerate(sorted(extended_vocab))}
idx2word = {idx: word for word, idx in word2idx.items()}

vocab_size_extended = len(extended_vocab)
embedding_matrix = np.zeros((vocab_size_extended, EMBEDDING_DIM))

print("Building embedding matrix...")

glove_dict = load_glove_embeddings()

for word in extended_vocab:
    idx = word2idx[word]
    # if word is in glove vocab, use glove vector
    if word in glove_dict:
        embedding_matrix[idx] = glove_dict[word]
    else:
        # use random vector for unknown words
        if word == UNK_TOKEN:
            embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))
        else:
            # remove the unknown word from the vocab
            # embedding_matrix[idx] = embedding_matrix[word2idx[UNK_TOKEN]]
            word2idx.pop(word)
            idx2word.pop(idx)

print("Embedding matrix built successfully.")

np.save(EMBEDDING_MATRIX_PATH, embedding_matrix)
print(f"Embedding matrix saved as '{EMBEDDING_MATRIX_PATH}'.")

with open(WORD2IDX_PATH, 'w', encoding='utf-8') as f:
    json.dump(word2idx, f, ensure_ascii=False, indent=4)
print(f"Mapping 'word2idx' saved as '{WORD2IDX_PATH}'.")

with open(IDX2WORD_PATH, 'w', encoding='utf-8') as f:
    json.dump(idx2word, f, ensure_ascii=False, indent=4)
print(f"Mapping 'idx2word' saved as '{IDX2WORD_PATH}'.")


Building embedding matrix...
Loading GloVe embeddings...


Repo card metadata block was not found. Setting CardData to empty.


Total GloVe words loaded: 400000
Embedding matrix built successfully.
Embedding matrix saved as './result/embedding_matrix.npy'.
Mapping 'word2idx' saved as './result/word2idx.json'.
Mapping 'idx2word' saved as './result/idx2word.json'.


For the second approach, we will use the FastText embeddings, which are trained on
subword information. This will help in encoding the OOV words as well.

In [ ]:
# Implementation of FastText for word embedding
from gensim.models import FastText

# Load Word2Vec model
w2v_model = EmbeddingMatrix.load()

# Create a list of words from the Word2Vec model
words = list(w2v_model.vocab)

# Create a FastText model with the same dimensions as the Word2Vec model
fasttext_model = FastText(size=w2v_model.dimension, window=5, min_count=1, workers=4)

# Build the FastText vocabulary
fasttext_model.build_vocab(words)

# Initialize embeddings with Word2Vec embeddings
for word in words:
    fasttext_model.wv[word] = w2v_model[word]


### Part 3.2 BiGru Model

The weight matrix will be created by BatchIterator class  based on the Glove word vectors we want to use and vocabularly word2index dictionary. For each word, index pair from word2index, the weight matrix at the position equals index will be set with the Glove vector that represents given word. 

For words that haven't been found in the Glove mapping, the random vector will be initialised. Then the weight matrix will be passed to the embedding layer as its parameters. 

It is very important to set the embedding_layer.weight.requires_grad = False, so that weights will not be updated during the training. After the GRU layer, extra features will be concatenated toehter with max-pooled, avg-pooled and last hidden size. The concatenated tensor will be next passed to the linear layer, so we have to set the input size to hidden_size * 3 + num_features.

In [21]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from common_utils import EmbeddingMatrix, UNK_TOKEN, EMBEDDING_DIM, tokenize
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
np.random.seed(42)
torch.manual_seed(0)

In [23]:
# Load word2vec embedding matrix
w2v_model = EmbeddingMatrix.load()
word2idx = w2v_model.word2idx

In [24]:
# Load datasets
from datasets import load_dataset 
# From Assignment PDF
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [55]:
import datasets 
import nltk 

def tokenize(texts, word2idx, max_len=512):
    tokenized = []
    lengths = []
    for text in texts:
        tokens = nltk.word_tokenize(text.lower())
        token_ids = [word2idx.get(word, word2idx[UNK_TOKEN]) for word in tokens]
        tokenized.append(torch.tensor(token_ids[:max_len], dtype=torch.int))
        lengths.append(len(token_ids))
    return tokenized, lengths

def prepare_dataset(dataset: datasets.DatasetDict, batch_size=50) -> torch.utils.data:
    set_tokenized, lengths = tokenize(dataset['text'], word2idx)
    
    set_tokenized = pad_sequence(set_tokenized, batch_first=True)

    lengths = torch.tensor(lengths, dtype=torch.long)

    set_labels = torch.tensor(dataset['label'])

    extra_features = torch.zeros((len(set_labels), 0), dtype=torch.float)

    set_data = data.TensorDataset(set_tokenized, extra_features, lengths, set_labels)

    return data.DataLoader(set_data, batch_size, shuffle=True)



In [56]:
train_loader = prepare_dataset(train_dataset)
val_loader = prepare_dataset(validation_dataset)
test_loader = prepare_dataset(test_dataset)

In [64]:
class BiGRU(nn.Module):
    """
    BiDirectional GRU neural network model with pre-trained word embeddings.

    Parameters
    -----------
    hidden_size: int 
        Number of features in the hidden state
    vocab_size: int
        The size of the vocabulary 
    n_extra_feat: int
        Number of additional features
    weight_matrix: numpy.ndarray 
        Matrix of pre-trained word embeddings (word_embedding)
    output_size: int
        Number of classes 
    n_layer: int, optional (default=1)
        Number of stacked recurrent layers. 
    dropout: float, optional (default=0.2)
        Probability of an element of the tensor to be zeroed.
    spatial_dropout: boolean, optional (default=True)
        Whether to use the spatial dropout. 
    bidirectional: boolean, optional (default=True)
        Whether to use the bidirectional GRU. 
    """

    def __init__(self, hidden_size, vocab_size, n_extra_feat, weights_matrix, output_size, n_layers=1, dropout=0.2, spatial_dropout=True, bidirectional=True):

        # Inherit everything from the nn.Module
        super(BiGRU, self).__init__()

        # Initialise attributes
        self.hidden_size = hidden_size 
        self.vocab_size = vocab_size 
        self.n_extra_feat = n_extra_feat
        self.weights_matrix = weights_matrix
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout
        self.spatial_dropout = spatial_dropout
        self.bidirectional = bidirectional
        self.n_directions = 2 if self.bidirectional else 1

        self.vocab_size, self.embedding_dim = self.weights_matrix.shape 
        # Embedding layer
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(weights_matrix, dtype=torch.float), freeze=False)

        self.dropout = nn.Dropout(self.dropout_p)
        if self.spatial_dropout:
            self.spatial_dropout1d = nn.Dropout1d(self.dropout_p)
        
        self.gru = nn.GRU(self.embedding_dim, self.hidden_size, num_layers=self.n_layers, dropout=(0 if n_layers == 1 else self.dropout_p), batch_first=True, bidirectional=self.bidirectional)

        # Linear layer input size is equal to hidden_size * 3 + n_extra_feat, becuase
        # we will concatenate max_pooling ,avg_pooling, last hidden state and additional features
        self.linear = nn.Linear(self.hidden_size * 3 + self.n_extra_feat, self.output_size)
    
    def forward(self, input_seq, input_feat, input_lengths, hidden=None):
        """Forward propagate through the neural network model.
        
        Parameters
        ----------
        input_seq: torch.Tensor
            Batch of input sequences.
        input_feat: torch.Tensor
            Batch of additional features.
        input_lengths: torch.LongTensor
            Batch containing sequences lengths.
        hidden: torch.FloatTensor, optional (default=None)
            Tensor containing initial hidden state.
            
        Returns
        -------
        torch.Tensor
            Logarithm of softmaxed input tensor.
        
        """
        # Extract batch_size
        self.batch_size = input_seq.size(0)
        
        # Embeddings shapes
        # Input: (batch_size,  seq_length)
        # Output: (batch_size, seq_length, embedding_dim)
        emb_out = self.embedding(input_seq)
        
        if self.spatial_dropout:
            # Convert to (batch_size, embedding_dim, seq_length)
            emb_out = emb_out.permute(0, 2, 1)
            emb_out = self.spatial_dropout1d(emb_out)
            # Convert back to (batch_size, seq_length, embedding_dim)
            emb_out = emb_out.permute(0, 2, 1)
        else:
            emb_out = self.dropout(emb_out)
        
        # Pack padded batch of sequences for RNN module
        packed_emb = nn.utils.rnn.pack_padded_sequence(emb_out, input_lengths, batch_first=True, enforce_sorted=False)
                
        # GRU input/output shapes, if batch_first=True
        # Input: (batch_size, seq_len, embedding_dim)
        # Output: (batch_size, seq_len, hidden_size*num_directions)
        # Number of directions = 2 when used bidirectional, otherwise 1
        # shape of hidden: (n_layers x num_directions, batch_size, hidden_size)
        # Hidden state defaults to zero if not provided
        gru_out, hidden = self.gru(packed_emb, hidden)
        # gru_out: tensor containing the output features h_t from the last layer of the GRU
        # gru_out comprises all the hidden states in the last layer ("last" depth-wise, not time-wise)
        # For biGRu gru_out is the concatenation of a forward GRU representation and a backward GRU representation
        # hidden (h_n) comprises the hidden states after the last timestep
        
        # Extract and sum last hidden state
        # Input hidden shape: (n_layers x num_directions, batch_size, hidden_size)
        # Separate hidden state layers
        hidden = hidden.view(self.n_layers, self.n_directions, self.batch_size, self.hidden_size)
        last_hidden = hidden[-1]
        # last hidden shape (num_directions, batch_size, hidden_size)
        # Sum the last hidden state of forward and backward layer
        last_hidden = torch.sum(last_hidden, dim=0)
        # Summed last hidden shape (batch_size, hidden_size)
        
        # Pad a packed batch
        # gru_out output shape: (batch_size, seq_len, hidden_size*num_directions)
        gru_out, lengths = nn.utils.rnn.pad_packed_sequence(gru_out, batch_first=True)
              
        # Sum the gru_out along the num_directions
        if self.bidirectional:
            gru_out = gru_out[:,:,:self.hidden_size] + gru_out[:,:,self.hidden_size:]
        
        # Select the maximum value over each dimension of the hidden representation (max pooling)
        # Permute the input tensor to dimensions: (batch_size, hidden, seq_len)
        # Output dimensions: (batch_size, hidden_size)
        max_pool = F.adaptive_max_pool1d(gru_out.permute(0,2,1), (1,)).view(self.batch_size,-1)
        
        # Consider the average of the representations (mean pooling)
        # Sum along the batch axis and divide by the corresponding lengths (FloatTensor)
        # Output shape: (batch_size, hidden_size)
        avg_pool = torch.sum(gru_out, dim=1) / lengths.view(-1,1).type(torch.FloatTensor) 

        # Concatenate max_pooling, avg_pooling, hidden state and input_feat tensor
        concat_out = torch.cat([last_hidden, max_pool, avg_pool, input_feat], dim=1)

        # concat_out = self.dropout(concat_out)
        out = self.linear(concat_out)
        return F.log_softmax(out, dim=-1)
  


  

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 8
vocab_size = w2v_model.vocab_size
n_extra_feat = 0
output_size = 2
n_layers = 1
dropout = 0.5 
learning_rate = 0.001
epochs = 40 
spatial_dropout = True

# Load the weights matrix
weights = np.load('result/embedding_matrix.npy')

# Create BiGru model 
model = BiGRU(hidden_size, vocab_size, n_extra_feat, weights, output_size, n_layers, dropout, spatial_dropout, bidirectional=True)
print(model)

# Create loss function
criterion = nn.NLLLoss()

# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model = model.to(device)


BiGRU(
  (embedding): Embedding(16164, 100)
  (dropout): Dropout(p=0.5, inplace=False)
  (spatial_dropout1d): Dropout1d(p=0.5, inplace=False)
  (gru): GRU(100, 8, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=24, out_features=2, bias=True)
)


In [66]:
def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in data_loader:
            sequences, features, lengths, labels = batch
            
            # Move tensors to device
            sequences = sequences.to(device)
            features = features.to(device)
            lengths = lengths.to(device)
            labels = labels.to(device)
            
            outputs = model(sequences, features, lengths)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            # Predictions
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(data_loader)
    accuracy = correct / total
    print(f'Validation Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')
    model.train()

In [67]:
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch in train_loader:
        sequences, features, lengths, labels = batch 

        # Move tensors to device (GPU or CPU)
        sequences = sequences.to(device)
        features = features.to(device)
        lengths = lengths.to(device)
        labels = labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass 
        outputs = model(sequences, features, lengths)

        # Compute loss
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    average_loss = epoch_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}')


Epoch 1/10, Loss: 0.7028
Epoch 2/10, Loss: 0.6665
Epoch 3/10, Loss: 0.6006
Epoch 4/10, Loss: 0.5205
Epoch 5/10, Loss: 0.4502
Epoch 6/10, Loss: 0.4020
Epoch 7/10, Loss: 0.3617
Epoch 8/10, Loss: 0.3246
Epoch 9/10, Loss: 0.2845
Epoch 10/10, Loss: 0.2520


In [68]:
evaluate(model, val_loader)

Validation Loss: 0.5055, Accuracy: 0.7824


# Question 3. Enhancement
(a) Report the accuracy score on the test set when the word embeddings are updated (Part 3.1).
   
(b) Report the accuracy score on the test set when applying your method to deal with OOV words
in Part 3.2.
   
(c) Report the accuracy scores of biLSTM and biGRU on the test set (Part 3.3).
   
(d) Report the accuracy scores of CNN on the test set (Part 3.4).
   
(e) Describe your final improvement strategy in Part 3.5. Report the accuracy on the test set
using your improved model.
   
(f) Compare the results across different solutions above and describe your observations with possible discussions.
